# DAS Microseismic Detection

### Written by Paige Given and Fantine Huot with Contributions from Ariel Lellouch, Bin Luo, Robert G. Clapp, Tamas Nemeth, Kurt Nihei, and Biondo L. Biondi

#### This notebook runs through the entire processing and prediction process for our ML workflow. 

## Load dependencies

In [ ]:
 cd ..

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import scipy
import math
from obspy.signal.filter import bandpass
import matplotlib.pyplot as plt
import enum
import os
from typing import List, Sequence, Text, Tuple
import logging
import os
import segyio

## Check for visible GPU devices
#### To maintain efficiency when working with large data, you will want to ensure you have a GPU connected

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.gpu_device_name()

In [ ]:
!nvidia-smi

## Set Datapath and Load in Data

#### Set datapath and file pattern for unprocessed data that you want to run predictions on:

In [ ]:
datapath = '/scratch/users/prgiven/'# replace me with datapath 
input_file_pattern = os.path.join(datapath, 'FORGE/', '*.sgy') # replace me with directory and file pattern           

#### Find filename and load data from numpy file

In [ ]:
filenames = sorted(tf.io.gfile.glob(input_file_pattern))
print('Found {} files'.format(len(filenames)))
print(filenames)

In [ ]:
with segyio.open(filenames[10], ignore_geometry=True) as f:
    data = segyio.tools.collect(f.trace[:])
print(data.shape)

##### Extract Time information from SEGY file

In [ ]:
from datetime import datetime, timedelta
np_file_path = '/scratch/users/prgiven/FORGE/SILIXA_78A-32_iDASv3-P13_220423144614_fieldID000011.sgy'
filename_time = os.path.basename(np_file_path)
time_str = filename_time.split('_')[-2]
start_time_str = f'20{time_str[:2]}-{time_str[2:4]}-{time_str[4:6]}_{time_str[6:8]}-{time_str[8:10]}-{time_str[10:12]}'

# Convert the start time to a datetime object
start_time = datetime.strptime(start_time_str, '%Y-%m-%d_%H-%M-%S')


In [ ]:
np_file_path = '/scratch/users/prgiven/FORGE2019_events/FORGE_78-32_iDASv3-P11_UTC190427172438.sgy'
filename_time = os.path.basename(np_file_path)
print(filename_time)
time_str = filename_time.split('UTC')[-1]
print(time_str)

#### Quality check by printing data and checking data shape

In [ ]:
print(data)

In [ ]:
data.shape

## Data pre-processing functions

**bp filter**: band-pass filters the data.\
**run prediction**: runs the model prediction on the data\
**preprocess**: runs bandpass on data, reshapes to model input size, clips amplitude values, and converts to float 32 format

In [ ]:
from scipy import signal

In [ ]:
def bpfilter(data,dt,bp_low,bp_high):
    sos = scipy.signal.iirfilter(N=15,Wn=[bp_low,bp_high],btype='bandpass',fs=1/dt,output='sos')
    return np.float32(scipy.signal.sosfilt(sos,data,axis=-1))

def hcfilter(data,dt,bp_high):
    sos = scipy.signal.iirfilter(N=15,Wn=bp_high,btype='lowpass',fs=1/dt,output='sos')
    return np.float32(scipy.signal.sosfilt(sos,data,axis=-1))

def normalize(data, axis=-1):
    stddev = np.std(data, axis=axis, keepdims=True)
    return np.divide(data, stddev, out=np.zeros_like(data), where=stddev != 0)

def decimate(data,axis=-1):
    data=signal.decimate(data,4,axis=axis)
    return np.float32(data)

def med(data,axis=0):
    data = data - np.median(data, axis=axis)
    return data

In [ ]:
def run_prediction(inputs, model):
    return model.predict(inputs, use_multiprocessing=True)

def preprocess(data):
    # crop it to input_height and width (cropping at the center)
    data=med(data,axis=0)
    data=hcfilter(data,0.002,100)
    data=normalize(data,axis=-1)
    data = data.reshape((1, 400, 250, 1)) # model input shape
    data = np.clip(data, -10, 10) # stats
    return np.float32(data)


### Look at Data

In [ ]:
plt.rcParams["figure.figsize"] = [20,10]
plt.ylabel('Depth (m)')
plt.xlabel('time (ms)')
plt.title('Raw Window - Full')
v=np.percentile(data,99)
plt.imshow((data),cmap='seismic')

In [ ]:
def preprocess2(data):
    # crop it to input_height and width (cropping at the center)
    data=med(data,axis=0)
    data=hcfilter(data,0.002,100)
    data=normalize(data,axis=-1)
    data = np.clip(data, -10, 10) # stats
    return np.float32(data)


In [ ]:
#remove top surface noise
data_cut=data[200:,:]
data_proccess=preprocess2(data_cut)

In [ ]:
plt.rcParams["figure.figsize"] = [20,10]
plt.ylabel('Depth (m)')
plt.xlabel('time (ms)')
plt.title('Processed Data')
plt.imshow((data_proccess),cmap='gray',vmin=-3, vmax=3)
plt.rcParams.update({'font.size': 12})
plt.ylim([1000,100])


## Load ML Model and Compile

In [ ]:
job_id = 'train_20210525_142709_cnn2d_modular_dec_baseline_' # Replace me
datapath2='/home/users/prgiven/DAS_ML/microseismic-detection-ml'
ckpt = '{}/models/{}/ckpt'.format(datapath2, job_id)
model = tf.keras.models.load_model(ckpt)
model.compile()

## Implement Sliding Windows, Pre-Procesing, and Running Prediction on Data

The cell below will create sliding windows from continuous data with window size (400,250). It the pre-processes the data, and runs the prediction on the processed windows. The final output is in the form [inputted,logits] where the inputted is the inputted window, and the logits is the result of the predictions.

In [ ]:
def sliding_window(data,start_time,time_step_size,distance_location,stride):
    n1=data.shape[0]
    w1=400
    d1=100 
    n2=data.shape[1]
    w2=250
    d2=245 

    num_windows1=len(range(0,n1-w1+1,d1))

    num_windows2=len(range(0,n2-w2+1,d2))

    logits=np.zeros((len(range(0,n1-w1+1,d1)),len(range(0,n2-w2+1,d2))))
    window_start_time1=[] 
    window_distance_location1=[]
    inputted=np.zeros((num_windows1,num_windows2,400,250))
    count1=0
    for i1 in range(0,n1-w1+1,d1):
        count2=0

        for i2 in range(0,n2-w2+1,d2):

            data_new=preprocess(data[i1:i1+w1,i2:i2+w2])
            window_start_time = start_time + timedelta(seconds=i2*time_step_size) + timedelta(milliseconds=i2*time_step_size*1000)
            window_start_time1 = np.append(window_start_time1,window_start_time) 

            window_distance_location = distance_location[i1 // stride[0]]
            window_distance_location1= np.append(window_distance_location1,window_distance_location)

            data_new.shape

            output = run_prediction(data_new, model)

            logits[count1,count2] = output 

            inputted[count1,count2]=np.squeeze(data_new)
            count2+=1
        count1+=1
    return inputted,logits,window_start_time1,window_distance_location1


    

In [ ]:
#%%time
probability_loop=[]
time_loop=[]
ap_loop=[]
for i in range(0,len(filenames)):
    with segyio.open(filenames[i], ignore_geometry=True) as f:
        data = segyio.tools.collect(f.trace[:])
        print(np.shape(data))
        
    #TIME INFO
    from datetime import datetime, timedelta
    np_file_path = filenames[i]
    filename_time = os.path.basename(np_file_path)
    time_str = filename_time.split('UTC')[-1]
    start_time_str = f'20{time_str[:2]}-{time_str[2:4]}-{time_str[4:6]}_{time_str[6:8]}-{time_str[8:10]}-{time_str[10:12]}'
    # Convert the start time to a datetime object
    start_time = datetime.strptime(start_time_str, '%Y-%m-%d_%H-%M-%S')
    time_step_size = 0.0005
    
    #SPACE INFO
    # Set the window size
    window_size = (400, 250)

    # Set the overlap size
    overlap_size = (300, 5)

    # Calculate the stride as the window size minus the overlap size
    stride = tuple(np.subtract(window_size, overlap_size))

    # Calculate the distance location for each window
    distance_location = np.arange(data.shape[0] - window_size[0] + 1, step=stride[0])
    #print(distance_location)

    
    data_cut=data[300:,:]

    [inputted,logits,window_start_time1,window_distance_location1]=sliding_window(data_cut,start_time,time_step_size,distance_location,stride) 
    apex=window_distance_location1+200
    probability = tf.nn.sigmoid(logits).numpy()
    positive_predictions=np.where(probability>0.5)
    neg_predictions=np.where(probability<0.1)
    #print(positive_predictions)
    positive_windows=inputted[positive_predictions[0],positive_predictions[1],:,:]
    neg_windows=inputted[neg_predictions[0],neg_predictions[1],:,:]
    positive_prob=probability[positive_predictions[0],positive_predictions[1]]
    positive_time=window_start_time1[positive_predictions[1]]
    positive_dist=window_distance_location1[positive_predictions[0]]
    np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/FORGE2019_RESULTS/' + str(i)+'_pos_logits.npy',positive_prob)
    np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/FORGE2019_RESULTS/' + str(i)+'_pos_windows.npy',positive_windows)
    np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/FORGE2019_RESULTS/' + str(i)+'_pos_times.npy',positive_time)
    np.save('/scratch/users/prgiven/continuous_data/continuous_processed/processed_data/FORGE2019_RESULTS/' + str(i)+'_pos_dist.npy',positive_dist)

    probability_loop=np.append(probability_loop,probability)
    time_loop=np.append(time_loop,window_start_time1)
    ap_loop=np.append(ap_loop,apex)
    print(filenames[i])
    print(i)
    #print(window_start_time)
    print("Total Windows",len(probability[probability>=0]))
    print("Number of Positive Windows with <50% Probability",len(probability[probability>0.5]))
   

In [ ]:
plt.figure(figsize=(13,5))
plt.hist(probability_loop,bins=50);
plt.yscale('log')
plt.rcParams.update({'font.size': 14})
plt.xlim([0,1])
plt.xlabel('Probability of Microseismic Event')
plt.ylabel('Number of Windows')
plt.title('Probability Distribution for FORGE')